## Introduction
This notebook provides an interactive environment for training a predictive model to estimate patient length of stay (LOS). This workflow is particularly suited for healthcare professionals and researchers interested in ICU resource planning and patient flow management.

Users are only requested to change the code block indicated with 

In [1]:
### ‼️User Action Required

### Package Installation
This code checks for the presence of required R packages and installs them if they are not already available.
These packages are essential for data preprocessing, model training, ensemble learning, and performance evaluation:
- `caret`: Core package for building and tuning predictive models.
- `caretEnsemble`: Allows combining multiple caret models into an ensemble for improved accuracy.
- `tidyverse`: Collection of packages for data manipulation, visualization, and general workflow.
- `MLmetrics`: Provides machine learning evaluation metrics (e.g., MAE, RMSE).
- `ranger`: Fast implementation of Random Forests, useful for training tree-based models efficiently.

⚠️ You only need to run this block once per session or when setting up a new environment.


In [2]:
import os
import pandas as pd
import pyreadr 
import numpy as np
import itertools
import warnings
import joblib
from sklearn.exceptions import NotFittedError
from sklearn.model_selection import train_test_split, cross_val_score, KFold, GridSearchCV, cross_val_predict 
from sklearn.feature_selection import VarianceThreshold
from sklearn.experimental import enable_iterative_imputer  
from sklearn.impute import IterativeImputer
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, make_scorer, mean_absolute_error, r2_score
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from scipy.stats import chi2_contingency, variation

### Load and Validate User Dataset

1. Please change the `data_path` the **path to the data** you want to train the model on. It can be in `.csv` or `.RData` format. If it's an `R.Data` file, please include the object name in the `object_name` variable.

2. If you want to include your **own predictors**, please change the `predictors` variable to include your a dataframe with one column stating the names of your predictors. If not, the the list of predictor variables used during the original model training is automatically loaded.

3. Add the **name** you want the model to be saved as to the `user_model_name` variable. the model will be saved as an .pkl file

⚠️ If not using your own predictors, make sure your dataset includes all required predictors listed in predictors.csv, as well as the target variable UnitLengthStay_trunc.

In [3]:
### ‼️User Action Required

data_path = "C:\\Users\\joana\\Documentos\\SLOS\\SLOS retraining\\SampledData.csv"
object_name = "sampled_data"
user_model_name = "YOUR_MODEL_NAME"
predictors = "C:\\Users\\joana\\Documentos\\SLOS\\SLOS retraining\\predictors.csv" # default predictors

In [5]:
if data_path.lower().endswith(".csv"):
    user_data = pd.read_csv(data_path)
else:
    raise ValueError("Unsupported file type. Please upload a .csv file.")

predictors_df = pd.read_csv(predictors)
predictors = predictors_df.iloc[:, 1].tolist()

if not all(p in user_data.columns for p in predictors):
    raise ValueError("Some required predictors are missing in your dataset.")

if 'UnitLengthStay_trunc' not in user_data.columns:
    raise ValueError("Target variable 'UnitLengthStay_trunc' is missing in the dataset.")

columns_to_select = predictors + ['UnitLengthStay_trunc'] + ['UnitLengthStay']
user_data = user_data[columns_to_select]

### Data Pre-processing
We remove zero and near-zero variance features, correlated predictors (for numeric and categorical features) and we impute missing data via the MICE algortihm

In [6]:
np.random.seed(998)
train_indices = train_test_split(
    range(len(user_data)), 
    train_size=0.8, 
    random_state=998,
    stratify=None  
)[0]
inTraining = train_indices

training = user_data.iloc[inTraining].copy()
training_dummy = training.copy()
testing = user_data.drop(user_data.index[inTraining]).copy()
testing_dummy = testing.copy()

In [9]:
def find_highly_correlated(df, threshold=0.75):
    corr_matrix = df.corr(method='pearson').abs()
    upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(bool))
    to_drop = [column for column in upper.columns if any(upper[column] > threshold)]
    return to_drop

numeric_cols = training.select_dtypes(include=[np.number]).columns.difference(["UnitLengthStay_trunc"])
high_corr_numeric = find_highly_correlated(training[numeric_cols])

train_data = training.drop(columns=high_corr_numeric)
test_data = testing.drop(columns=high_corr_numeric)

In [10]:
def cramers_v(x, y):
    confusion_matrix = pd.crosstab(x, y)
    if confusion_matrix.shape[0] < 2 or confusion_matrix.shape[1] < 2:
        return 0
    chi2 = chi2_contingency(confusion_matrix)[0]
    n = confusion_matrix.sum().sum()
    phi2 = chi2 / n
    r, k = confusion_matrix.shape
    phi2corr = max(0, phi2 - ((k - 1)*(r - 1)) / (n - 1))
    rcorr = r - ((r - 1)**2) / (n - 1)
    kcorr = k - ((k - 1)**2) / (n - 1)
    return np.sqrt(phi2corr / min((kcorr - 1), (rcorr - 1)))

cat_cols = train_data.select_dtypes(include='object').columns
cramer_matrix = pd.DataFrame(index=cat_cols, columns=cat_cols)

for col1, col2 in itertools.combinations(cat_cols, 2):
    val = cramers_v(train_data[col1], train_data[col2])
    cramer_matrix.loc[col1, col2] = val
    cramer_matrix.loc[col2, col1] = val
np.fill_diagonal(cramer_matrix.values.astype(float), 0)

high_corr_cat = []
for col in cramer_matrix.columns:
    if any(cramer_matrix[col].astype(float) > 0.5):
        high_corr_cat.append(col)

train_data = train_data.drop(columns=high_corr_cat)
test_data = test_data.drop(columns=high_corr_cat)

In [11]:
def mice_imputation(df, target_col='UnitLengthStay_trunc', random_state=100, max_iter=5):
    df_imp = df.copy()

    numeric_cols = df_imp.select_dtypes(include=[np.number]).columns.tolist()
    if target_col in numeric_cols:
        numeric_cols.remove(target_col)
    
    categorical_cols = df_imp.select_dtypes(include=['object', 'category']).columns.tolist()

    if numeric_cols:
        numeric_imputer = IterativeImputer(
            estimator=RandomForestRegressor(random_state=random_state, n_estimators=10),
            max_iter=max_iter,
            random_state=random_state
        )
        df_imp[numeric_cols] = numeric_imputer.fit_transform(df_imp[numeric_cols])

    for col in categorical_cols:
        mode_value = df_imp[col].mode()
        if len(mode_value) > 0:
            df_imp[col] = df_imp[col].fillna(mode_value[0])
    
    return df_imp

In [12]:
np.random.seed(100)
training_imp = mice_imputation(training.drop(columns=['UnitLengthStay_trunc']), random_state=100)
training_imp['UnitLengthStay_trunc'] = training_dummy['UnitLengthStay_trunc']
training = training_imp.copy()

np.random.seed(100)
testing_imp = mice_imputation(testing.drop(columns=['UnitLengthStay_trunc']), random_state=100)
testing_imp['UnitLengthStay_trunc'] = testing_dummy['UnitLengthStay_trunc']
testing = testing_imp.copy()

### Model Traning
This section covers the full training pipeline, from splitting the data to building an ensemble model using sklearn.ensemble.
 
Two base models are trained: Linear regression (sklearn.linear_model) and Random Forest (sklearn.ensemble).

A stacked ensemble model is built from the base learners.The final stacked model is saved as "stacked_best_model", and all of the models are saved in the .pkl file.

In [13]:
X_train = training.drop(columns=['UnitLengthStay_trunc'])
y_train = training['UnitLengthStay_trunc']
X_test = testing.drop(columns=['UnitLengthStay_trunc'])
y_test = testing['UnitLengthStay_trunc']

categorical_cols = X_train.select_dtypes(include=['object', 'category']).columns.tolist()
numerical_cols = X_train.select_dtypes(include=[np.number]).columns.tolist()

preprocessor = ColumnTransformer(
    transformers=[
        ('num', 'passthrough', numerical_cols),  # Keep numerical columns as-is
        ('cat', OneHotEncoder(drop='first', sparse_output=False, handle_unknown='ignore'), categorical_cols)
    ],
    remainder='passthrough'
)

X_train_processed = preprocessor.fit_transform(X_train)
X_test_processed = preprocessor.transform(X_test)

feature_names = []
feature_names.extend(numerical_cols)
if categorical_cols:
    cat_encoder = preprocessor.named_transformers_['cat']
    cat_feature_names = cat_encoder.get_feature_names_out(categorical_cols)
    feature_names.extend(cat_feature_names)

X_train_processed = pd.DataFrame(X_train_processed, columns=feature_names)
X_test_processed = pd.DataFrame(X_test_processed, columns=feature_names)

cv_folds = KFold(n_splits=5, shuffle=True, random_state=42)
rmse_scorer = make_scorer(lambda y_true, y_pred: np.sqrt(mean_squared_error(y_true, y_pred)), 
                         greater_is_better=False)

C:\Users\joana\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\preprocessing\_encoders.py:246: UserWarning: Found unknown categories in columns [1] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(


In [14]:
lm_model = LinearRegression()
lm_cv_scores = cross_val_score(lm_model, X_train_processed, y_train, 
                              cv=cv_folds, scoring=rmse_scorer, verbose=1)
lm_model.fit(X_train_processed, y_train)
lm_rmse = -lm_cv_scores.mean()

In [15]:
rf_param_grid = {
    'n_estimators': [100],  # Fixed for initial training
    'max_features': list(range(5, min(11, len(feature_names)+1))),  # mtry equivalent, adjusted for actual feature count
    'min_samples_split': [10],  # Approximation of min.node.size
    'random_state': [42]
}

rf_model = RandomForestRegressor()
rf_grid_search = GridSearchCV(
    rf_model, 
    rf_param_grid, 
    cv=cv_folds, 
    scoring=rmse_scorer,
    verbose=1,
    n_jobs=-1
)
rf_grid_search.fit(X_train_processed, y_train)
rf_best_model = rf_grid_search.best_estimator_
rf_rmse = -rf_grid_search.best_score_

Fitting 5 folds for each of 6 candidates, totalling 30 fits


In [16]:
model_list = {
    'lm': lm_model,
    'rf': rf_best_model
}

lm_pred_train = cross_val_predict(lm_model, X_train_processed, y_train, cv=cv_folds)
rf_pred_train = cross_val_predict(rf_best_model, X_train_processed, y_train, cv=cv_folds)

stacking_features = pd.DataFrame({
    'lm_pred': lm_pred_train,
    'rf_pred': rf_pred_train
})

stacked_param_grid = {
    'n_estimators': [100],
    'max_features': [2],  # mtry = 2
    'min_samples_split': [10, 20, 30, 40],  # min.node.size equivalent
    'criterion': ['squared_error'],  # variance equivalent
    'random_state': [42]
}

stacked_model = RandomForestRegressor()
stacked_grid_search = GridSearchCV(
    stacked_model,
    stacked_param_grid,
    cv=cv_folds,
    scoring=rmse_scorer,
    verbose=1,
    n_jobs=-1
)

stacked_grid_search.fit(stacking_features, y_train)
stacked_best_model = stacked_grid_search.best_estimator_
stacked_rmse = -stacked_grid_search.best_score_

models = {
    'individual_models': model_list,
    'stacked_model': stacked_best_model,
    'stacking_features_columns': stacking_features.columns.tolist()
}

if user_model_name[-4] != ".pkl":
    user_model_name += ".pkl"
joblib.dump(models, user_model_name)
print("Models saved to ", user_model_name)


Fitting 5 folds for each of 4 candidates, totalling 20 fits
Models saved to  YOUR_MODEL_NAME.pkl


### Model Prediction and Evaluation
This section handles making predictions with the trained stacked model and evaluating its performance using key metrics.

1. the `predict_stacked` call generates **predictions** for the test set using the stacked model.

2. We **evaluate** the trained model performance on three metrics:

- Root Mean Squared Error (RMSE): Measures the average magnitude of the prediction errors.

- Mean Absolute Error (MAE): Measures the average of the absolute errors, giving a sense of how far off the predictions are.

- R-squared (R2): Indicates the proportion of the variance in the dependent variable that is predictable from the independent variables.

These metrics are computed using the functions available in the sklearn package.

In [17]:
def predict_stacked(X, models, preprocessor):
        
    X_processed = preprocessor.transform(X)
    
    lm_pred = models['individual_models']['lm'].predict(X_processed)
    rf_pred = models['individual_models']['rf'].predict(X_processed)
    
    stacking_input = pd.DataFrame({
        'lm_pred': lm_pred,
        'rf_pred': rf_pred
    })
    
    final_pred = models['stacked_model'].predict(stacking_input)
    return final_pred

In [18]:
test_predictions = predict_stacked(X_test, models, preprocessor)
test_rmse = np.sqrt(mean_squared_error(y_test, test_predictions))
test_mae = mean_absolute_error(y_test, test_predictions)
test_r2 = r2_score(y_test, test_predictions)

print(f"RMSE: {test_rmse:.4f}")
print(f"MAE: {test_mae:.4f}")
print(f"R²: {test_r2:.4f}")

RMSE: 0.5264
MAE: 0.3819
R²: 0.9880


C:\Users\joana\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\preprocessing\_encoders.py:246: UserWarning: Found unknown categories in columns [1] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(
C:\Users\joana\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but LinearRegression was fitted with feature names
  warnings.warn(
C:\Users\joana\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but RandomForestRegressor was fitted with feature names
  warnings.warn(
